In [1]:
#import bibliotek
pip install mysql-connector-python

import mysql.connector as sql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np

#Połączenie z bazą danych
username = 'nick'
password = 'haslo'
host = 'localhost'
database = 'airlines'
port = port

#Tworzenie połączenia z bazą
url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = sqlalchemy.create_engine(url)



#Zaczytanie ramki flight_df
flight_df = pd.read_csv(r"../data/raw/flight_df_01.csv")

#Wczytanie tabeli aircraft
aircraft_df = read_sql_table('aircraft')

#Usunięcie kolumny number_of_seats
aircraft_df.drop('number_of_seats', axis=1, inplace=True)

#Usunięcie kolumny id
aircraft_df.drop('id', axis=1, inplace=True)

#Usunięcie duplikatów
aircraft_df = aircraft_df.drop_duplicates()

# Sprawdzenie, czy istnieją duplikaty dla kolumny 'tail_num' z różnymi rokami produkcji
duplicates = aircraft_df[aircraft_df.duplicated(subset=['tail_num'], keep=False)]
print(duplicates)

aircraft_df_duplicated = duplicates.copy()

#Sprawdzenie, czy w ramkce aircraft_df występują duplikaty wewnątrz kolumny tail_num
aircraft_df_is_duplicated = aircraft_df.duplicated(subset='tail_num')
aircraft_df_duplicated = aircraft_df.loc[aircraft_df_is_duplicated]

#sortowanie i usuwanie duplikatów (pozostaje tylko nowszy rok)
aircraft_df.sort_values(by=['tail_num', 'manufacture_year'], ascending=False, inplace=True)
aircraft_df.drop_duplicates(subset=['tail_num'], keep='first', inplace=True)

#Sprawdzenie
test_tail = 'N783CA'
test_value = aircraft_df.loc[aircraft_df['tail_num'] == test_tail]['manufacture_year']
test_value = int(test_value.iloc[0])  # Wybieramy pierwszy element serii danych

expected_value = 2000
assert test_value == expected_value, f"For 'tail_num' == '{test_tail}' expected {expected_value} received {test_value}"



# Złączenie ramek danych flight_df i aircraft_df
tmp_flight_df = pd.merge(flight_df, aircraft_df, on='tail_num', how='left')
tmp_flight_df.info()

#Sprawdzenie czy są duplikaty
tmp_flight_df[tmp_flight_df.duplicated()]

#Nadpisanie flight_df
flight_df = tmp_flight_df



# Opóźnienia a rok produkcji samolotu

#Grupowanie - zależność roku produkcji do częstotliwości opóźnień
delays_by_manufacture_year_df = flight_df.groupby('manufacture_year')['is_delayed'].mean()

#Generowanie wykresu dla delays_by_manufacture_year_df
plt.figure(figsize=(15, 10))
plt.plot(delays_by_manufacture_year_df.index, delays_by_manufacture_year_df.values, marker = 'o', linestyle='--')
plt.title('Average delays by aircraft manufacture year', fontweight='bold')
plt.xlabel('Manufacture Year')
plt.ylabel('Average Delays')
plt.grid(alpha=0.5)
plt.show()

#Filtracja
flights_per_year = flight_df['manufacture_year'].value_counts()
flights_per_year

years_over_10000_flights = flights_per_year[flights_per_year > 10000].index
years_over_10000_flights 

filtered_flight_df = flight_df[flight_df['manufacture_year'].isin(years_over_10000_flights)]

#Grupowanie - zależność roku produkcji do częstotliwości opóźnień
filtered_delays_by_manufacture_year_df = filtered_flight_df.groupby('manufacture_year')['is_delayed'].mean()

#Generowanie wykresu
plt.figure(figsize=(15, 10))
plt.plot(filtered_delays_by_manufacture_year_df.index, filtered_delays_by_manufacture_year_df.values, marker='o', linestyle='--')
plt.title('Average delays by aircraft manufacture year', fontweight='bold')
plt.xlabel('Manufacture Year')
plt.ylabel('Average Delays')
plt.grid(alpha=0.5)
plt.show()

#Dodanie kolumny `manufacture_year_agg` do ramki `flight_df`

# Definiowanie przedziałów co 3 lata
bins = list(range(int(flight_df['manufacture_year'].min()), int(flight_df['manufacture_year'].max()) + 4, 3))

# Tworzenie etykiet przedziałów
labels = [f"{bins[i]}-{bins[i]+2}" for i in range(len(bins)-1)]

# Dodawanie kolumny manufacture_year_agg z niestandardowymi etykietami przedziałów
flight_df['manufacture_year_agg'] = pd.cut(flight_df['manufacture_year'], bins=bins, labels=labels, right=False)

# Ustawianie NaN w kolumnie manufacture_year_agg tam, gdzie manufacture_year jest NaN
flight_df.loc[flight_df['manufacture_year'].isna(), 'manufacture_year_agg'] = np.nan



#Stworzenie zmiennej `flight_delays_by_manufacture_year_agg_df`

# Obliczenie średnich opóźnień po agregacji kolumny manufacture_year_agg
flight_delays_by_manufacture_year_agg_df = flight_df.groupby('manufacture_year_agg')['is_delayed'].mean().reset_index()
flight_delays_by_manufacture_year_agg_df

#Generowanie wykresu w oparciu o dane zawarte w flight_delays_by_manufacture_year_agg_df
plt.figure(figsize=(15, 10))
plt.plot(flight_delays_by_manufacture_year_agg_df['manufacture_year_agg'], flight_delays_by_manufacture_year_agg_df['is_delayed'], marker='o', linestyle='--')
plt.title('Average delays by aircraft manufacture year', fontweight='bold')
plt.xlabel('Manufacture Year')
plt.ylabel('Average Delays')
plt.grid(alpha=0.5)
plt.show()



#Wyznaczenie TOP 5 roczników produkcji

# Grupowanie 
manufacture_year_stats = flight_df.groupby('manufacture_year').agg(num_flights=('tail_num', 'count'), mean_delay=('is_delayed', 'mean'))
# Sortowanie według liczby lotów w malejącej kolejności
manufacture_year_top5 = manufacture_year_stats.sort_values(by='num_flights', ascending=False).head()
manufacture_year_top5

#Zapisanie ramki danych
flight_df.to_csv(r"../data/processed/flight_df_02.csv", index=False)